In [1]:
from dotenv import load_dotenv
from langchain_openai import OpenAI
load_dotenv()

True

In [27]:
llm = OpenAI(
    temperature=0,
    model_name='gpt-3.5-turbo-instruct'
)

In [11]:
from langchain.chains import LLMMathChain
from langchain.agents import Tool

ll_math = LLMMathChain(llm=llm)

math_tool = Tool(
    name='Calculator',
    func=ll_math.run,
    description='Useful for when you need to answer questions about math.'
)

tools = [math_tool]

/home/miller/.local/share/virtualenvs/lang_chain_studies-WWOOpBwl/lib/python3.10/site-packages/langchain/chains/llm_math/base.py:58: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [12]:
print(tools[0].name, '---', tools[0].description)

Calculator --- Useful for when you need to answer questions about math.


In [13]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent='zero-shot-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=10
)

/home/miller/.local/share/virtualenvs/lang_chain_studies-WWOOpBwl/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [14]:
zero_shot_agent('What is root over 25')

/home/miller/.local/share/virtualenvs/lang_chain_studies-WWOOpBwl/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
 I should use a calculator to find the square root of 25
Action: Calculator
Action Input: square root of 25
Observation: Answer: 5.0
Thought: I now know the final answer
Final Answer: 5.0

> Finished chain.


{'input': 'What is root over 25', 'output': '5.0'}

In [15]:
problem = """
    You are building a house. There are two bedrooms of 5 meters by 5 meters each, 
    a open kitchenwith 7 meters by 6 meters 
    and balcony of 3 meters by 2 meters.
    Qhat is the total area of your house? 
"""

zero_shot_agent(problem)



> Entering new AgentExecutor chain...
 To find the total area, we need to calculate the area of each room and then add them together.
Action: Calculator
Action Input: Calculate 5*5
Observation: Answer: 25
Thought: Now we need to calculate the area of the second bedroom.
Action: Calculator
Action Input: Calculate 5*5
Observation: Answer: 25
Thought: Next, we need to calculate the area of the open kitchen.
Action: Calculator
Action Input: Calculate 7*6
Observation: Answer: 42
Thought: Lastly, we need to calculate the area of the balcony.
Action: Calculator
Action Input: Calculate 3*2
Observation: Answer: 6
Thought: Now we have all the individual areas, we can add them together to find the total area.
Action: Calculator
Action Input: Add 25+25+42+6
Observation: Answer: 98
Thought: I now know the final answer.
Final Answer: The total area of the house is 98 square meters.

> Finished chain.


{'input': '\n    You are building a house. There are two bedrooms of 5 meters by 5 meters each, \n    a open kitchenwith 7 meters by 6 meters \n    and balcony of 3 meters by 2 meters.\n    Qhat is the total area of your house? \n',
 'output': 'The total area of the house is 98 square meters.'}

In [32]:
from langchain.tools import DuckDuckGoSearchRun


search = DuckDuckGoSearchRun()

search_tool = Tool.from_function(
    func=search.run,
    name='Search',
    description='useful when you need to search the internet for information'
)

tools = [search_tool, math_tool]

agent = initialize_agent(
    tools,
    llm,
    agent='zero-shot-react-description',
    verbose=True
)

In [34]:
agent.run('Get Microsoft stock price taken from google finance and display in both USD and INR values')



> Entering new AgentExecutor chain...
 I should use the Search tool to find the stock price information.
Action: Search
Action Input: 'Microsoft stock price google finance'
Observation: Discover real-time Microsoft Corporation Common Stock (MSFT) stock prices, quotes, historical data, news, and Insights for informed trading and investment decisions. Stay ahead with Nasdaq. View Microsoft Corporation MSFT stock quote prices, financial information, real-time forecasts, and company news from CNN. Market Cap. $3,172B. Today's Change. (0.34%) $1.46. Current Price. $426.73. Price as of July 29, 2024, 4:00 p.m. ET. It was more about what the company isn't doing about a recent crisis, rather ... Get a real-time Microsoft Corporation (MSFT) stock price quote with breaking news, financials, statistics, charts and more. The Case for Investing in Google Stock Google, or Alphabet, stock has gained more than 40% in the past year, putting it on par with Microsoft.
Thought: I should use the Calculat

'The Microsoft stock price in USD is $426.73 and in INR is 2240332.5.'

## Create a custom tool

In [38]:
from langchain_core.output_parsers import JsonOutputParser
from langchain.prompts import PromptTemplate
from langchain.tools import tool

@tool('lower_case', return_direct=True)
def StructuredResponseTool(question: str):
    '''Use this tool to send a prompt and get a JSON returned with three fields - Topic, Question_Details and Detailed_Response'''

    json_prompt = PromptTemplate.from_template(
        '''Return a JSON objetc with an `answer` key that answers the following question: {question}.
        The JSON object will have fields - Topic, Question_Details asn Detailed_Response
        '''
    )

    model = OpenAI(model_name='gpt-3.5-turbo-instruct', temperature=0.5)

    json_parser = JsonOutputParser()

    json_chain = json_prompt | model | json_parser

    result = json_chain.invoke({'question': question})

    return result

tools = [StructuredResponseTool]

zero_shot_agent = initialize_agent(
    agent='zero-shot-react-description',
    tools=tools,
    llm=llm,
    verbose= True,
    max_iterations=10
)

zero_shot_agent.run(input='Which is the topt footbel team in the world at the moment?')



> Entering new AgentExecutor chain...
 I should use lower_case to get the JSON response
Action: lower_case
Action Input: Which is the topt footbel team in the world at the moment?
Observation: {'Topic': 'Soccer', 'Question_Details': 'Which is the top football team in the world at the moment?', 'Detailed_Response': 'The top football team in the world at the moment is currently debated among experts and fans, but many consider FC Barcelona to be the current best team due to their recent success in major tournaments and their star players such as Lionel Messi and Neymar Jr.'}


> Finished chain.


{'Topic': 'Soccer',
 'Question_Details': 'Which is the top football team in the world at the moment?',
 'Detailed_Response': 'The top football team in the world at the moment is currently debated among experts and fans, but many consider FC Barcelona to be the current best team due to their recent success in major tournaments and their star players such as Lionel Messi and Neymar Jr.'}